# Read and examine the business data set

For starters I need to get a basic understanding of the data.  It is a **open data set of businesses** in San Diego.  It was obtained from government sources.  I was given an [excel (xlsx) file](https://drive.google.com/file/d/1bsExDRghMkyBrbEuuC1p4VMuna7lBhhH/view).  

- Saved the `data worksheet` from xlsx file as csv (it's just easier)
- Use `pandas` to read and maniputate the dataframe
- First step is understand the data (shape, dtypes, info(), etc)

The `output` from this processing is a new geo dataframe for further, problem specific analysis.

In [1]:
import pandas as pd
import geopandas as gpd
from geopandas import GeoDataFrame
from shapely.geometry import Point
import osmnx as ox

### Read the data

In [2]:
biz_df = pd.read_csv('../data/biz_data.csv', sep='\t', index_col=0)

### Examine layout and structure of the dataframe

In [3]:
biz_df.shape

(53675, 18)

In [4]:
biz_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 53675 entries, 1 to 53675
Data columns (total 18 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   BUSINESS ACCT#     53675 non-null  int64 
 1   DBA NAME           53675 non-null  object
 2   OWNERSHIP TYPE     53675 non-null  object
 3   ADDRESS            53675 non-null  object
 4   CITY               53647 non-null  object
 5   ZIP                53675 non-null  object
 6   STATE              53675 non-null  object
 7   BUSINESS PHONE     48269 non-null  object
 8   OWNER NAME         53675 non-null  object
 9   CREATION DT        53675 non-null  object
 10  START DT           53675 non-null  object
 11  EXP DT             53675 non-null  object
 12  NAICS              53675 non-null  int64 
 13  ACTIVITY DESC      53675 non-null  object
 14  today              53675 non-null  object
 15  years              53675 non-null  int64 
 16  naics_code         53675 non-null  int64

In [5]:
biz_df.isnull().sum()

BUSINESS ACCT#          0
DBA NAME                0
OWNERSHIP TYPE          0
ADDRESS                 0
CITY                   28
ZIP                     0
STATE                   0
BUSINESS PHONE       5406
OWNER NAME              0
CREATION DT             0
START DT                0
EXP DT                  0
NAICS                   0
ACTIVITY DESC           0
today                   0
years                   0
naics_code              0
NAICS Description       0
dtype: int64

In [6]:
print("missing business phone numbers: {:.2%}".format(biz_df['BUSINESS PHONE'].isnull().sum() / len(biz_df)))

missing business phone numbers: 10.07%


# Examine, rework, and extend dtypes

Next some basic analysis of dtypes.  There are two types in the data set, int64 and object (str).  The column with the most missing values is the phone number for now that's ok.  The specific values I want to transform are:

1. NAICS - This is an int in the input source. We need a character representation

2. ZIP - this value is really zip+4 as object (i.e. str)
  
3. Dates - There are three date variables.  They are strings in the input data.  It's easy to convert to py datetime so ...

4. Geocode - Using osmnx api to osm

## NAICS

  - Convert to str
  - For starters, use the first to char's for sector (will worry about more detail later)

**TBD: add NAICS reference to explain levels**

In [7]:
naics_desc = pd.read_excel('../data/2017_NAICS_Descriptions.xls')

naics_desc['Code'] = naics_desc['Code'].astype(str)

lookup_dict = naics_desc[['Code', 'Title']].set_index('Code')['Title'].to_dict()

def lookup(key):
    if key in ('48', '49'):
        key = '48-49'
    if key in ('44', '45'):
        key = '44-45'
    if key in ('31', '32', '33'):
        key = "31-33"
        
    desc = lookup_dict[key]
    if desc[-1:] == 'T':
        desc = desc[:-1]
        
    return desc.rstrip()

In [8]:
biz_df['sector'] = biz_df['NAICS'].astype(str).apply(lambda x: x[:2])

In [9]:
biz_df['sector_desc'] = biz_df['sector'].apply(lambda x: lookup(x) + f"({x})")

In [10]:
biz_df.sector.value_counts()

54    11358
81     8159
62     4170
56     4125
23     4122
45     3567
72     2951
44     2855
53     2730
42     1719
48     1624
61     1499
71     1257
52     1029
33      727
51      650
32      263
49      256
31      247
55      202
11       96
22       58
92        9
21        2
Name: sector, dtype: int64

In [11]:
biz_df.sector_desc.value_counts(normalize=True) #* 100

Professional, Scientific, and Technical Services(54)                            0.211607
Other Services (except Public Administration)(81)                               0.152007
Health Care and Social Assistance(62)                                           0.077690
Administrative and Support and Waste Management and Remediation Services(56)    0.076851
Construction(23)                                                                0.076796
Retail Trade(45)                                                                0.066456
Accommodation and Food Services(72)                                             0.054979
Retail Trade(44)                                                                0.053190
Real Estate and Rental and Leasing(53)                                          0.050862
Wholesale Trade(42)                                                             0.032026
Transportation and Warehousing(48)                                              0.030256
Educational Services(

### So you see what I did with the transformed data?

  - sector is used for selection
  - sector_desc is used for human consumption (you need to look at the excel referened above)

## ZIP

  - Notice the ZIP column is really ZIP+4
  - Createing new column called zip_code
  - Easy string hack to get the actual zip

In [12]:
biz_df['zip_code'] = biz_df['ZIP'].apply(lambda x: x.split('-')[0].strip())

In [13]:
biz_df.iloc[27]

BUSINESS ACCT#                                   2007006934
DBA NAME                     10717 SORRENTO VALLEY ROAD LLC
OWNERSHIP TYPE                                          LLC
ADDRESS                            10717 SORRENTO VALLEY RD
CITY                                              SAN DIEGO
ZIP                                              92121-1610
STATE                                                    CA
BUSINESS PHONE                               (858) 452-2200
OWNER NAME                   10717 SORRENTO VALLEY ROAD LLC
CREATION DT                                        6/7/2007
START DT                                           6/1/2004
EXP DT                                            5/31/2021
NAICS                                                  5313
ACTIVITY DESC             ACTIVITIES RELATED TO REAL ESTATE
today                                            12/22/2020
years                                                    14
naics_code                              

In [14]:
biz_92110 = biz_df.query(f"zip_code == '92110'").reset_index()

In [15]:
len(biz_92110)

1941

## ZIP codes around El Cajon Boulevard BID

This section will filter the larger biz_df to include only those in zip codes related to the Blvd BID. <br/>
After a quick look at a map I see two zip codes in the area.<br/>

These query examples us the query method from pandas.  I've noticed some issues running this type of query in collab.

In [16]:
zips = ['92115', '92116']

In [17]:
ecb_df = biz_df.query(f"zip_code in @zips")

In [18]:
len(ecb_df)

2355

In [22]:
len(ecb_df.query(f"zip_code == '92115'"))

1387

In [23]:
len(ecb_df.query(f"zip_code == '92116'"))

968

In [26]:
print(f"{len(ecb_df) / len(biz_df):.2%} of the businesses are in this area.")

4.39% of the businesses are in this area.


In [21]:
ecb_df.columns

Index(['BUSINESS ACCT#', 'DBA NAME', 'OWNERSHIP TYPE', 'ADDRESS', 'CITY',
       'ZIP', 'STATE', 'BUSINESS PHONE', 'OWNER NAME', 'CREATION DT',
       'START DT', 'EXP DT', 'NAICS', 'ACTIVITY DESC', 'today', 'years',
       'naics_code', 'NAICS Description', 'sector', 'sector_desc', 'zip_code'],
      dtype='object')

## Datetime

It appears datetime types not supported in ESRI driver for shape files, so just leave this alone for now.<br/>

For now it's enough to know that this is how you can convert the string to datetime.  <br/>

I guess if one wants to do some geo-temporal analysis it can just be done on the other side in the geodf.

In [ ]:
biz_df.columns

In [ ]:
biz_df['CREATION DT'] = pd.to_datetime(biz_df['CREATION DT'])
biz_df['START DT'] = pd.to_datetime(biz_df['START DT'], errors='coerce')
biz_df['EXP DT'] = pd.to_datetime(biz_df['EXP DT'])

In [ ]:
biz_df.info()

In [ ]:
# build the ecb df again with transformed columns
ecb_df = biz_df.query(f"zip_code in @zips")

In [ ]:
ecb_df.info()

## Gecode

Geocoding uses the Nominatim service of OSM via the osmnx api.  The terms of use state one request per second so the wrapper function handles that.<br/>

If you're executing this code be aware of the timing of things and the size of the dataframe you're trying to geocode. <br/>

I have a few examples below

In [27]:
def good_address(addr):
    if addr.find('SUITE') > 0:
        return addr[:addr.find('SUITE')-1]
    else:
        return addr

In [28]:
import time
def geocode_address(row):
    """
    Special function applied to a zip_code transformed row.  
    
    Notes:
      1. nominatim terms of use require one query per sec so we sleep on each iteration.
      2. When we get no match, returning None so we can query for
         invalid geo's later.
    """
    time.sleep(1)
    good_addr = good_address(row.ADDRESS)
    geocode_query = f"{good_addr}, {row.CITY}, {row.STATE}, {row.zip_code}"
    try:
        lat, lon = ox.geocode(geocode_query)
        return Point(lon, lat)
    except:
        print(geocode_query)
        return None #Point(lon, lat).wkt

In [37]:
# An example for row 27
row = ecb_df.iloc[27]
query = f"{good_address(row.ADDRESS)}, {row.CITY}, {row.STATE}, {row.zip_code}"
print("Address to geocode: " + query)


geocode_address(biz_df.iloc[27]).wkt

Address to geocode: 4745 EL CAJON BLVD, SAN DIEGO, CA, 92115


'POINT (-117.219997 32.899025)'

### Back-of-the-envelope calculation

In [38]:
ecb_df.zip_code.value_counts()

92115    1387
92116     968
Name: zip_code, dtype: int64

In [39]:
#back of the envelope calc for time to compute - This simple hack estimates the minutes required to geocode.
len(ecb_df) / 60.

39.25

**So little over 39 minutes to geocode 92115 and 92116**

**You can stop here as I've previously done this and saved the gdf**

In [ ]:
#%%timeit
ecb_df['geometry'] = ecb_df.apply(lambda r: geocode_address(r), axis=1)

In [ ]:
ecb_gdf = GeoDataFrame(ecb_df, geometry=ecb_df.geometry)

In [ ]:
ecb_gdf.geometry.isnull().sum()

In [ ]:
ecb_gdf.to_file("../data/ecb.shp")

### so the step above that I did not execute saved the geocoded file.  I'm just reading it back in.

In [42]:
previously_computed_gdf = gpd.read_file("../data/ecb.shp")

In [47]:
denominator = len(previously_computed_gdf)
numerator = previously_computed_gdf['geometry'].isnull().sum()

print(f"Total businesses: {denominator}")
print(f"Total that Nominatim couldn't find: {numerator}")
print(f"Unable to geocode: {numerator/denominator:.2%} addresses")

Total businesses: 2355
Total that Nominatim couldn't find: 18
Unable to geocode: 0.76% addresses


### That's not bad actually.

**Can anyone suggest an approach to clean the addresses?**

## Final Note on Geocoding

I am currently creating a personal Nominatim server.  I won't have to throttle it so I will geocode and publish the entire file.